# Introduction to ipyautoui

ipyautoui is a Python package that provides a simple way to create interactive user interfaces in Jupyter notebooks. It is built on top of Jupyter widgets and Pydantic and provides a higher-level API for creating interactive widgets.  It is particularly well suited for creating settings panels.


## TO DO IN THIS NOTEBOOK
- [ ] Add a brief description of the package and its history?
- [ ] Show how to use ipyautoui to create a simple settings panel relatively quickly
  - [ ] Mention the active development of the package and the fact that it is still a bit buggy
  - [ ] Mention other features of Pydantic that may be relevant to our dashboard
- [ ] Have them replace existing settings widgets with control widget generated with ipyautoui
- [ ] Connect new control widget to dashboard